In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii


In [10]:
bounds = pickle.load(open('./pickle/bounds.pk','rb'))

In [11]:
master_cat = pd.read_csv('./catdata/master_catalog_jan_2023.csv')

In [12]:
cat_files = ['cat1_50.pk','cat51_100.pk','cat101_150.pk','cat151_200.pk','cat201_235.pk',
             'cat236_257.pk','cat258_279.pk','cat280_320.pk','cat321_360.pk','cat361_406.pk']

In [13]:
object_matches = pickle.load(open('./pickle/matches_delta006_crowding300.pk','rb'))

In [14]:
train = pickle.load(open('./pickle/training_data_d006_c300.pk','rb'))

In [15]:
def load_cat(field): # change to match-case
    if field <= 50: to_load = cat_files[0]
    elif field <= 100: to_load = cat_files[1]
    elif field <= 150: to_load = cat_files[2]
    elif field <= 200: to_load = cat_files[3]
    elif field <= 235: to_load = cat_files[4]
    elif field <= 257: to_load = cat_files[5]
    elif field <= 279: to_load = cat_files[6]
    elif field <= 320: to_load = cat_files[7]
    elif field <= 360: to_load = cat_files[8]
    elif field <= 406: to_load = cat_files[9]
    print(f'Loading {to_load} ...')
    catalogue = pickle.load(open(f'./pickle/{to_load}','rb'))
    
    return catalogue

## Generate Training Data

In [16]:
def generate_training_data(matches):
    cat = load_cat(1)
    
    columns = ['obj_id','class','i','g','di','dg','ra','dec','field','pdidx']
    values = []
    object_ids = []
    
    for field in matches: # iterate through each field ID
        print(f'Working on {field}')
        working_field = matches[field] # take the list of matches
        if field not in cat: # load the correct catalogue
            cat = load_cat(field)
        for m in working_field: # iterate through each match and grab values from catalogues
            obj_id = master_cat.loc[m[1]].ID
            class_ = master_cat.loc[m[1]].CLASS #STR?
            
            if obj_id in object_ids: continue # if we've already added the object then skip
            else: object_ids.append(obj_id)   # (due to duplicated)
            
            if class_ == 1: class_str = 'gc'
            elif class_ == 4: class_str = 'galaxy'
            elif class_ == 6: class_str = 'star'
            else: continue # skip non-gc/gal/stars
            
            row = cat[field][m[0]]
            ra = row['RA']
            dec = row['Dec']
            g = row['g']
            i = row['i']
            dg = row['dg']
            di = row['di']
            
            values.append([obj_id,class_str,i,g,di,dg,ra,dec,field,m[0]])
    
    training_data_dict = dict(zip(columns,zip(*values)))
    training_data_df = pd.DataFrame(training_data_dict)
    return training_data_df

In [ ]:
training_data = generate_training_data(object_matches)

In [ ]:
with open('./pickle/training_data_d006_c300.pk','wb') as f:
    pickle.dump(training_data,f)

### Analyse Matches

In [ ]:
#subset_matches = {k: object_matches[k] for k in (5,11,13,14)}

In [ ]:
set(training_data['class'])

In [ ]:
a = [ [1,10], [2,20], [3,30] ]
c = ['dig','ten']

In [ ]:
d = dict(zip(c,a))

In [ ]:
dict(zip(c,zip(*a)))